# 第4回講義 演習

## 目次
1. PyTorch概観
2. Tensor  
    2.1. Tensorとは  
    2.2. Tensorの生成  
    2.3. Tensorの操作  
    2.4. Tensorの微分  
3. Dataset / DataLoader  
    3.1. Dataset  
    3.2. DataLoader  
4. モデルの定義  
    4.1. モデルパーツ一覧  
    4.2. Sequentialモデル   
    4.3. Subclassingモデル  
5. モデルの学習  
    5.1. 損失関数の設定  
    5.2. オプティマイザの設定  
    5.3. 学習モード / 推論モード  
    5.4. 学習の実行

In [ ]:
from IPython.display import clear_output

!pip install torchvision==0.6.1
clear_output()

import math
from pprint import pprint

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torchsummary import summary
import torchvision
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

random_seed = 42
np.random.seed(random_seed)
torch.manual_seed(random_seed)

## 1. PyTorch概観

PyTorchでは、基本的に以下の流れに沿ってモデルを構築します。

1. 学習データの設定
2. ネットワークの設定
3. 損失関数/オプティマイザの設定
4. 学習の実行
5. 予測の実行

以下に、ORデータに対するロジスティック回帰の例を示します。

In [ ]:
# ORデータの作成
x_data = np.array([[0, 1], [1, 0], [0, 0], [1, 1]]).astype(np.float32)
t_data = np.array([[1], [1], [0], [1]]).astype(np.float32)

In [ ]:
# Step 1. 学習データの設定
x = torch.tensor(x_data, dtype=torch.float)
t = torch.tensor(t_data, dtype=torch.float)

# Step 2. ネットワークの設定
net = nn.Linear(in_features=2, out_features=1, bias=True)

# Step 3. 損失関数の設定
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1)

# Step 4. 学習の実行
for epoch in range(1000):
    optimizer.zero_grad()  # 勾配の初期化
    y = net(x)   #  予測の計算（順伝播）
    loss = loss_fn(y, t)  # 損失関数の計算
    loss.backward()  # 勾配の計算
    optimizer.step()  # パラメータの更新
    if (epoch + 1) % 100 == 0:
        print('EPOCH: {}, Train Cost, {:.3f}'.format(epoch + 1, loss))

# Step 5. 予測の実行
y = net(x)
# 正解データの[1,1,0,1]に近い値が得られていることを確認する
torch.sigmoid(y)

## 2. Tensor

### 2.1. Tensorとは
PyTorchでは主にTensor型の配列を扱います。先ほどの例ではxやtがそれにあたります。  
Tensor型は、NumPyの配列であるndarray型のデータとほぼ同様に扱うことができます。  
ndarrayとの相違点は、GPU上の演算が可能であること、勾配情報を保持できることです。

In [ ]:
x = torch.tensor([[0, 1], [1, 0], [0, 0], [1, 1]],
                 dtype=torch.float)

print(x)
print(type(x))

データ型には、主に以下の二種類を用います。  
- torch.float: float32
- torch.long: int64

In [ ]:
print('小数:', torch.float)
print('整数:', torch.long)

### 2.2. Tensorの生成  
Tensorの生成方法には、場合に応じて以下のような分類があります。
- torch.tensor() : 既存のデータから生成する場合
- torch.* : サイズだけが決まっている場合
- torch.*_like : 既存のTensorと同じサイズ・同じデータ型で生成する場合
- tensor.new_* : 既存のTensorと異なるサイズ・同じデータ型で生成する場合  

ここでは上の二つについて扱います。  
先ほどの例では既存のndarrayから、torch.tensor()を用いてTensorを生成しました。  
torch.from_numpy()を用いても同様に作成できますが、copyではなくviewとなります。  
同様に、既存のリストからtorch.tensor()を用いてTensorを生成することができます。

In [ ]:
# 既存のndarrayからTensorを生成
x_data = np.array([[0, 1], [1, 0], [0, 0], [1, 1]])
x = torch.tensor(x_data, dtype=torch.float)
print('inputs(Tensor): \n', x)
print()

# Tensorからndarrayへの変換
x_array = x.numpy()
print('inputs(ndarray): \n', x_array)
print()

# 既存のリストからTensorを生成
t_data = [[1], [1], [0], [1]]
t = torch.tensor(t_data, dtype=torch.float)
print('outputs(Tensor): \n', t)
print()

# Tensorからlistへの変換
t_list = t.tolist()
print('outputs(list): \n', t_list)

所望のサイズだけが決まっている場合は、次のようにtorch.*でTensorを生成することができます。

In [ ]:
# torch.Tensor()で初期化されたTensorを生成
a = torch.Tensor(2, 3)
print('torch.Tensor(): \n', a)
print()

# torch.zeros()でゼロ埋めされたTensorを生成
b = torch.zeros(3, 4, dtype=torch.int64)
print('torch.zeros(): \n', b)
print()

# torch.ones()で一埋めされたTensorを生成
c = torch.ones(2, 3, 4, dtype=torch.float32)
print('torch.ones(): \n', c)
print()

# torch.eye()で単位行列のTensorを作成
d = torch.eye(3)
print('torch.eye(): \n', d)

乱数で初期化されたTensorは、以下のように生成することができます。

In [ ]:
# torch.rand()で一様乱数からTensorを生成
e = torch.rand(2, 3)
print('torch.rand(): \n', e)
print()

# torch.randn()で標準正規乱数からTensorを生成
f = torch.randn(2, 3)
print('torch.randn(): \n', f)
print()

# torch.randperm()でランダムな順列からTensorを生成
g = torch.randperm(10)
print('torch.randperm(): \n', g)

numpy-likeに数列を生成することも可能です。

In [ ]:
# torch.arange()で、numpy.arange()と同様にTensorを生成
h = torch.arange(0, 1, 0.2)
print('torch.arange(): \n', h)
print()

# torch.linspace()で、numpy.linspace()と同様にTensorを生成
i = torch.linspace(0, 10, 5)
print('torch.linspace(): \n', i)

### 2.3. Tensorの操作
GPU上で計算を行う際には、TensorをGPUに乗せる必要があります。  
CPUとGPUの切り替えには、以下のようにtoメソッドを用います。

In [ ]:
x = torch.tensor([[0, 1], [1, 0], [0, 0], [1, 1]],
                 dtype=torch.float)

# tensor.deviceで、device(CPUかGPUか)の確認
x = x.to('cpu')
print('現在のdevice:', x.device)
print()

# tensor.to('cuda:0')で、GPUに切り替え
x = x.to('cuda:0')
print('GPUに切替:', x.device)

# tensor.to('cpu')で、CPUに切り替え
x = x.to('cpu')
print('CPUに切替:', x.device)
print()

# torch.cuda.is_available()で、GPUが利用可能か確認
print('GPUが利用可能かどうかチェック:', torch.cuda.is_available())

# GPUが利用可能ならGPUに切り替え、そうでないならCPUに切り替え
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
x = x.to(device)
print('GPUが利用可能ならGPUに切替:', x.device)

Tensorはndarrayと同様にスライスすることができます。

In [ ]:
x = torch.tensor([[0, 1], [1, 0], [0, 0], [1, 1]],
                 dtype=torch.float)

print('スライス前のTensor: \n', x)
print()

print('2列目を切り出し: \n', x[:, 1])
print()

print('最終行を切り出し: \n', x[-1, :])
print()

print('2行目までを切り出し: \n', x[:2, :])
print()

print('1行おきに切り出し: \n', x[::2, :])

Tensorの情報は、以下のように取得できます。

In [ ]:
x = torch.tensor([[0, 1], [1, 0], [0, 0], [1, 1]],
                 dtype=torch.float)

# tensor.deviceでdeviceを取得
print('tensor.device: ', x.device)

# tensor.dtypeでデータ型を取得
print('tensor.dtype : ', x.dtype)

# tensor.shapeで形状を取得（tensor.size()でも同様）
print('tensor.shape : ', x.shape)

# tensor.dim()で次元を取得
print('tensor.dim() : ', x.dim())

Tensorの結合は、以下のように実行できます。

In [ ]:
x1 = torch.zeros(2, 3)
x2 = torch.ones(2, 3)

# torch.cat()で既存の軸方向に結合（元データと次元は変わらない）
x_cat = torch.cat([x1, x2], dim=0)
print('縦方向に結合: \n', x_cat)
print('Tensorの次元: ', x_cat.dim())
print('Tensorのサイズ:', x_cat.size(), '\n')


# torch.stack()で新規の軸方向に結合（元データより次元が大きくなる）
x_stack = torch.stack([x1, x2], dim=0)
print('新規の軸方向に結合: \n', x_stack)
print('Tensorの次元：', x_stack.dim())
print('Tensorのサイズ', x_stack.size())

Tensorの変形は、以下のように実行できます。

In [ ]:
x3 = torch.arange(0, 12)
print('変形前のTensor: \n', x3)
print()

# tensor.reshape()で、指定した形状に変形
x3 = x3.reshape(1, 3, 4)
print('tensor.reshape(): \n', x3)
print()

# tensor.transpose()で指定した軸を交換
x3 = x3.transpose(1, 2)
print('tensor.transpose(): \n', x3)
print()

# tensor.squeeze()で要素数が1の次元を削除
x3 = x3.squeeze()
print('tensor.squeeze(): \n', x3)
print()

# tensor.unsqueeze()で指定した次元を追加
x3 = x3.unsqueeze(1)
print('tensor.unsqueeze(): \n', x3)

Tensorの分割は、以下のように実行できます。

In [ ]:
x4 = torch.arange(0, 12)
x4 = x4.reshape(3, 4)
print('分割前のTensor: \n', x4)
print()

# torch.chunk()で指定した個数の塊に分割
x_chunks = torch.chunk(x4, 2, dim=0)
print('torch.chunk(): \n', x_chunks)
print()

# torch.split()で指定した個数ごとに分割
x_splits = torch.split(x4, 2, dim=1)
print('torch.splits(): ')
pprint(x_splits)

### 2.4. Tensorの微分
深層学習での計算、特に誤差逆伝播法では計算グラフという概念が用いられます。  
計算グラフとは、複数の関数からなる計算過程をグラフとして表したものです。  
計算グラフでは、各演算ノード間の勾配が計算され、保持される必要があります。  
PyTorchでは、以下のようにTensor型がこの勾配の計算と保持を実現できます。  
内部的には主にautogradパッケージによって自動微分機能が提供されています。

In [ ]:
# 一次関数 y=wx+b を例に自動微分を実行

# requires_grad=Trueとして自動微分の対象を指定
x = torch.tensor(2.0, requires_grad=True)
w = torch.tensor(3.0, requires_grad=True)
b = torch.tensor(4.0, requires_grad=True)

# 計算グラフを構築
y = w*x + b

# tensor.backward()で勾配を計算
# 計算グラフを遡って、requires_grad=TrueとしたTensorに勾配の情報を付与
y.backward()

# tensor.gradでその変数による偏微分値を表示
print('dy/dx(expected to be 3): ', x.grad)  # dy/dx = w = 3.
print('dy/dw(expected to be 2): ', w.grad)  # dy/dw = x = 2.
print('dy/db(expected to be 1): ', b.grad)  # dy/db = 1.

深層学習では、損失関数を重みやバイアスで偏微分した値が主に用いられます。  
初めに見たロジスティック回帰の例では、BCEに対し勾配を計算していました。  
ここではナイーブな勾配降下法を例にとって、最適化の挙動を見てみましょう。

- optim.SGD : (最適化する係数，その他オプション)を入力として与える．

In [ ]:
# Step 1. 学習データの設定
x = torch.randn(5, 3)
y = torch.randn(5, 2)

# Step 2. ネットワークの設定
linear = nn.Linear(3, 2)

# Step 3. 損失関数・オプティマイザの設定
criterion = nn.MSELoss() # 平均二乗誤差を損失関数として採用
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01) # 確率的勾配降下法（オンライン学習）で学習

# Step 4. 学習の実行(1 epoch)
optimizer.zero_grad()
pred = linear(x)  # 予測の計算（順伝播）
loss = criterion(pred, y)  # 損失関数の計算
loss.backward()  # 勾配の計算（逆伝播）
print('損失関数の微分値:')
print('dL/dw:', linear.weight.grad)  # 重みに関する偏微分
print('dL/db:', linear.bias.grad)  # バイアスに関する偏微分
print()

print('手書きの最急降下法:')  # 手書きの最急降下法
print(linear.weight.sub(0.01 * linear.weight.grad))
print()
print(linear.bias.sub(0.01 * linear.bias.grad))
print()

print('実装済みの最急降下法:')  # 実装済みの最急降下法
optimizer.step()
print(linear.weight)
print(linear.bias)  # 結果が一致することを確認

## 3. Dataset / DataLoader
深層学習における入力データの扱いには、以下のようなステップがあります。
1. 読み込み
2. 前処理
3. ラベル付け
4. シャッフル
4. 分割

PyTorchでは1から3をDatasetが、4と5をDataLoaderがそれぞれ実現します。

### 3.1. Dataset
Datasetは与えられたデータを読み込んで前処理を施し、ラベルを付与します。  
ここでは後でも取り上げるMNISTデータを例に、その扱いをみてみましょう。

In [ ]:
# GPUの利用可否を取得
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# MNISTのデータセットを取得
mnist = fetch_openml('mnist_784', data_home='/root/userspace/public/day1/chap02/data')
X = mnist.data.astype('float32')
y = mnist.target.astype('int64')

# 学習データの値域を[0,1]に正規化
X /= 255

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/7, shuffle=False)

# データを変形（画像数， チャネル数， 高さ， 幅）
X_train = X_train.reshape(60000, 1, 28, 28)
X_test = X_test.reshape(10000, 1, 28, 28)

# Tensorに変換
X_train = torch.tensor(X_train, device=device, dtype=torch.float32)
X_test = torch.tensor(X_test, device=device, dtype=torch.float32)
y_train = torch.tensor(y_train, device=device, dtype=torch.int64)
y_test = torch.tensor(y_test, device=device, dtype=torch.int64)

# データセットを作成
dataset_train = TensorDataset(X_train, y_train)
dataset_test = TensorDataset(X_test, y_test)

torch.utils.data.Datasetを継承することで、Datasetを自作することもできます。  
Datasetを自作する際には、\_\_len\_\_と\_\_getitem\_\_を定義する必要があります。  
- \_\_len\_\_: データセットの長さを返すメソッド  
- \_\_getitem\_\_: 説明変数と目的変数の組を一つ返す

In [ ]:
# torch.utils.data.Datasetを継承し、自作データセットを作成
class MNISTDataset(torch.utils.data.Dataset):
    
    def __init__(self, transform=None):
        # MNISTデータのダウンロード
        self.mnist = fetch_openml('mnist_784', data_home='/root/userspace/public/day1/chap02/data')
        self.data = self.mnist.data.reshape(-1, 28, 28).astype('uint8')  # 説明変数
        self.target = self.mnist.target.astype('int64')  # 目的変数
        self.transform = transform  # 前処理
        
    def __len__(self):
        return len(self.data)  # データセットの長さを返す
    
    def __getitem__(self, idx):
        data = self.data[idx]
        target = torch.from_numpy(np.array(self.target[idx]))
        
        if self.transform:
            data = self.transform(data)
                
        sample = (data, target)
        return sample  # 説明変数と目的変数の組を一つ返す

MNISTのような有名なデータセットは、予め用意されているものを利用できます。  
この場合、torchvision.transformsパッケージを以下のように用いて前処理します。

In [ ]:
# 画像データの前処理を定義
trans = transforms.Compose([
    transforms.ToTensor(),  # Tensorに変換
    transforms.Normalize((0.5, ), (0.5, ))  # 平均0.5, 標準偏差0.5に正規化
])

# ダウンロードをしてから，mnistディレクトリに展開
mnist_train = MNIST('/root/userspace/public/day1/chap04/data/mnist', train=True, download=True, transform=trans)
mnist_test = MNIST('/root/userspace/public/day1/chap04/data/mnist', train=False, download=True, transform=trans)

Datasetは以下のようにtorch.utils.data.random_split()を用いて分割できます。

In [ ]:
# torch.utils.data.random_split()によって訓練データと検証データに分割
mnist_train, mnist_valid = torch.utils.data.random_split(mnist_train, [48000, 12000])

### 3.2. DataLoader
DataLoaderを用いることで、Datasetをシャッフルし、ミニバッチに分割することができます。

In [ ]:
# 訓練データのDataLoaderを作成
loader_train = DataLoader(mnist_train, 
                          shuffle=True,  # データをシャッフルする
                          batch_size=1024,  # ミニバッチの大きさ: 1024(＊ただし最後の１つはあまり)
                          num_workers=4  # コア数: 4
                         )

# 検証データのDataLoaderを作成
loader_valid = DataLoader(mnist_valid,
                          shuffle=True,  # データをシャッフルする
                          batch_size=1024,  # ミニバッチの大きさ: 1024
                          num_workers=4  # コア数: 4
                         )

# テストデータのDataLoaderを作成
loader_test = DataLoader(mnist_test,
                         shuffle=True,  # データをシャッフルする
                         batch_size=1024,  # ミニバッチの大きさ: 1024
                         num_workers=4  # コア数: 4
                        )

作成したDataLoaderからは、次のようにしてミニバッチごとにデータを取り出すことができます。


`shuffle=True` と指定したため，毎回違う順番になります．

In [ ]:
for data in loader_train:
    inputs, labels = data
    break
print('ミニバッチの大きさ: ', len(labels))
print('ミニバッチの中身:', labels)

## 4. モデル定義
PyTorchにおけるモデル定義には、以下の二つの方法があります。  
- Sequentialモデル
- Subclassingモデル

まずはPyTorchにおいてよく用いられる深層学習モデルのパーツを見てみます。  


### 4.1. モデルパーツ一覧

PyTorchにおける深層学習モデルのパーツには、以下のようなものがあります。  
（以下に挙げる例の中には、まだ講義で扱っていない内容も含まれます。）
- 全結合層 : torch.nn.Linear
- ドロップアウト : torch.nn.Dropout
- 畳み込み層(2次元) : torch.nn.Conv2d
- 最大プーリング(2次元) : torch.nn.MaxPool2d
- バッチ正規化(2次元) : torch.nn.BatchNorm2d
- RNN層 : torch.nn.RNN
- LSTM層 : torch.nn.LSTM
- シグモイド関数 : torch.nn.Sigmoid
- tanh関数 : torch.nn.Tanh
- ReLU関数 : torch.nn.ReLU

In [ ]:
# 全結合層
m = nn.Linear(20, 30)
input = torch.randn(128, 20)
output = m(input) 
print('全結合層の出力の大きさ:', output.shape) # (128 * 20) * (20 * 30)なので．．．
print()

# ドロップアウト
m = nn.Dropout(p=0.4) # 40%の値を0にする．
input = torch.randn(3, 5)
output = m(input)
print('ドロップアウト層の出力: \n', output)
print()

# ReLU関数
m = nn.ReLU()
input = torch.tensor([1.2, -0.8])
output = m(input)
print('ReLU関数の入力: ', input)
print('ReLU関数の出力: ', output)

### 4.2. Sequentialモデル
torch.nn.Sequentialクラスを用いるSequentialモデルは、以下のようにして各層を順々に積み重ねていくものです。  
この記法はKerasのSequentialモデルAPIに準じています。直感的な記述が可能ですが、単純なモデルしか組めません。

In [ ]:
### モデルのインスタンスを作成
model = nn.Sequential()

### add_module()で層を追加
model.add_module('fc1', nn.Linear(2, 5))
model.add_module('sigmoid', nn.Sigmoid())
model.add_module('fc2', nn.Linear(5, 1))

### 作成したモデルの情報を表示
summary(model.to(device), (1, 2))

### 4.3. Subclassingモデル
もう一つは、torch.nn.Moduleクラスのサブクラスとして定義する記法です。  
本講座では、より汎用性の高いこちらの記法をメインとして採用します。  
Subclassingモデルにも、以下のように大きく分けて二種類の記法があります。
- パラメタを持つものは\_\_init\_\_に、持たないものはforwardに書く
- パラメタを持たない活性化関数なども含めて全て\_\_init\_\_に書く  

これら二つにもそれぞれ長所と短所があり、使い分けられるのが望ましいです。  
まず前者の実装例を見てみましょう。利点は以下のようなものが挙げられます。
- 各層のパラメタの有無が明確になる
- モデルを簡潔に記述することができる  

この場合、活性化関数はtorch.nn.functional(as F)のものを使います。  
例えばReLU関数はnn.ReLUではなく、F.reluをforward内に記述します。

In [ ]:
# nn.Moduleクラスを継承してクラス定義
# パラメタを持つものは__init__に、持たないものはforwardに書く
class Model1(nn.Module):
    def __init__(self):
        super(Model1, self).__init__()
        self.fc1 = nn.Linear(2, 5)
        self.fc2 = nn.Linear(5, 1)
        
    # 順伝播の定義
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# モデルのインスタンスを作成
model1 = Model1()

# 作成したモデルの情報を表示
summary(model1.to(device), (1, 2))

次に後者の実装例を見てみましょう。利点は、以下のようなものが挙げられます。
- printやsummaryで活性化関数なども表示できる
- 学習/推論モードがdropoutなどに自動で反映される

In [ ]:
# nn.Moduleクラスを継承してクラス定義
# パラメタを持たない活性化関数なども含めて全て__init__に書く
class Model2(nn.Module):
    def __init__(self):
        super(Model2, self).__init__()
        self.fc1 = nn.Linear(2, 5)
        self.ac1 = nn.ReLU()
        self.fc2 = nn.Linear(5, 1)
        
    # 順伝播の定義
    def forward(self, x):
        x = self.fc1(x)
        x = self.ac1(x)
        x = self.fc2(x)
        return x

# モデルのインスタンスを作成
model2 = Model2()

# 作成したモデルの情報を表示
summary(model2.to(device), (1, 2))

## nn.Sequentialの使い方の例
深層学習の火付け役となったAlexNetの実装を見てみましょう。このようにいくつかの記法を組み合わせることも可能です。  
この実装例のようにいくつかの記法を組み合わせて記述することで、よりコンパクトで可読性の高いモデル定義ができます。

In [ ]:
class AlexNet(nn.Module):

    def __init__(self, num_classes=1000):
        super().__init__()
        # 畳み込みブロックのSequentialによる定義
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        # 全結合ブロックのSequentialによる定義
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
    
alexnet = AlexNet()

summary(alexnet.to(device), (3, 224, 224))

## 5. 学習
### 5.1. 損失関数の設定
PyTorchでは、torch.nnパッケージで主要な損失関数が実装されているので、これを用います。  
- 回帰
    - nn.MSELoss(平均二乗誤差)
    - nn.L1Loss(平均絶対誤差)
- 2クラス分類
    - nn.BCELoss(2クラス交差エントロピー)
    - nn.BCEWithLogitsLoss(ロジット2クラス交差エントロピー)
- 多クラス分類
    - nn.CrossEntropyLoss(多クラス交差エントロピー)

In [ ]:
input = torch.randn(1000)
target = torch.randn(1000)

mse_fn = nn.MSELoss()
mse = mse_fn(input, target)
print('平均二乗誤差:', mse)


input = torch.randn(3, 5)
target = torch.randperm(3, dtype=torch.long)

cel_fn = nn.CrossEntropyLoss()
cel = cel_fn(input, target)
print('交差エントロピー誤差:', cel)

### 5.2. オプティマイザの設定
PyTorchでは、torch.optimパッケージ内に主要なオプティマイザが実装されています。
- torch.optim.SGD
- torch.optim.Adagrad
- torch.optim.Adadelta
- torch.optim.Adam

「2.4. Tensorの微分」で見たナイーブな最急降下法の例を、改めて見ておきましょう。

In [ ]:
# Step 1. 学習データの設定
x = torch.randn(5, 3)
y = torch.randn(5, 2)

# Step 2. ネットワークの設定
linear = nn.Linear(3, 2)

# Step 3. 損失関数・オプティマイザの設定
criterion = nn.MSELoss()
rate = 0.01
optimizer = torch.optim.SGD(linear.parameters(), lr=rate)

# Step 4. 学習の実行(1 epoch)
pred = linear(x)  # 予測の計算（順伝播）
loss = criterion(pred, y)  # 損失関数の計算
loss.backward()  # 勾配の計算（逆伝播）
print('損失関数の微分値:')
print('dL/dw:', linear.weight.grad)  # 重みに関する偏微分
print('dL/db:', linear.bias.grad)  # バイアスに関する偏微分
print('')

print('手書きの最急降下法: ')  # 手書きの最急降下法
print(linear.weight.sub(rate * linear.weight.grad))
print(linear.bias.sub(rate *linear.bias.grad))
print('')

print('実装済みの最急勾配法: ')  # 実装済みの最急勾配法
optimizer.step()
print(linear.weight)
print(linear.bias)  # 結果が一致することを確認

### 5.3 学習モード / 推論モード
　PyTorchでは、以下の二つのモードがあります。
- 学習モード
- 推論モード

学習モードと推論モードの大きな違いはパラメータ更新の有無です。  
学習モードではパラメータが更新されますが、推論モードではパラメータが更新されません。

また学習モードと推論モードで、ドロップアウト(torch.nn.Dropout)の挙動も変わります。  
学習モードではドロップアウトが有効ですが、推論モードでは自動で無効となります。  
ただしtorch.nn.functional.dropoutでは自動で切り替わらないので、注意する必要があります。

In [ ]:
# モデルを作成
model = nn.Sequential()

# 学習モードか推論モードか確認(model.trainingがTrueなら学習モード、Falseなら推論モード)
print('初期モード: ', model.training)

# 推論モードに切替
model.eval()
print('推論モードに切替後: ', model.training)

# 学習モードに切替
model.train()
print('学習モードに切替後: ', model.training)

推論段階ではパラメータを更新しないため、計算グラフを構築する必要がありません。  
計算グラフを構築しないよう設定することによって、メモリを節約することができます。  
この設定は、with torch.no_grad()内で対象となる処理を記述することで実現できます。

In [ ]:
x = torch.tensor(3.0, dtype=torch.float32, requires_grad=True)

# torch.no_grad()との比較用
y = x ** 2
y.backward()
print('dy/dx(expected to be 6.0): ', x.grad)

In [ ]:
x = torch.tensor(3.0, dtype=torch.float32, requires_grad=True)

# 計算グラブを構築しないため、backward()でエラーが発生する
with torch.no_grad():
    y = x ** 2
    y.backward()
    x.grad

### 5.4. 学習の実行
これまでに扱った内容を駆使して、MNISTデータのMLPによる分類を実装してみます。
1. 学習データの設定
2. ネットワークの設定
3. 損失関数/オプティマイザの設定
4. 学習の実行
5. 予測の実行

以下では中間層が一層だけのMLPをバッチサイズ1024、エポック数10で学習します。

＜備考＞
- Net classはnn.Moduleを継承するclass．したがって，`python super().__init__()`によってnn.Moduleもinitする必要がある．
- 精度向上のために，ハイパーパラメーターを調節してみると良い．



In [ ]:
# Step 1. 学習データの設定
# 前処理の設定
trans = transforms.Compose([
    transforms.ToTensor(),  # Tensorに変換
    transforms.Normalize((0.5, ), (0.5, ))  # 平均0.5, 標準偏差0.5に正規化
])

# Datasetの作成
ds_train = MNIST('/root/userspace/public/day1/chap04/data/mnist', train=True, download=True, transform=trans)
ds_test = MNIST('/root/userspace/public/day1/chap04/data/mnist', train=False, download=True, transform=trans)
ds_train, ds_valid = torch.utils.data.random_split(ds_train, [48000, 12000])

# DataLoaderの作成
batch_size = 1024  # バッチサイズ: 1024
dl_train = DataLoader(ds_train, batch_size=batch_size, shuffle=True, num_workers=4)
dl_valid = DataLoader(ds_valid, batch_size=batch_size, shuffle=True, num_workers=4)
dl_test = DataLoader(ds_test, batch_size=batch_size, shuffle=True, num_workers=4)


# Step 2. ネットワークの設定
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(28 * 28, 200)  # 入力層
        self.l2 = nn.Linear(200, 200)  # 中間層
        self.l3 = nn.Linear(200, 10)  # 出力層
    
    def forward(self, x):
        x = x.reshape(-1, 28 * 28)  # 二次元の画像データを一次元のベクトルに変形
        x = F.relu(self.l1(x))  # 活性化関数は全てReLU
        x = F.relu(self.l2(x))
        x = self.l3(x)
        return x
    
# GPUが利用可能ならGPUを使用
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = Net().to(device)


# Step 3. 損失関数/オプティマイザの設定
loss_fn = nn.CrossEntropyLoss()  # 損失関数: 交差エントロピー誤差
optimizer = optim.SGD(net.parameters(), lr=0.1)  # オプティマイザ: 最急降下法


# Step 4. 学習の実行
num_epochs = 10  # エポック数: 10

for epoch in range(num_epochs):
    train_loss, train_acc, val_loss, val_acc = 0.0, 0.0, 0.0, 0.0
    
    net.train()  # 学習モードに切替
    for inputs, labels in dl_train:
        inputs, labels = inputs.to(device), labels.to(device)  # データをGPUに乗せる
        optimizer.zero_grad()  # 勾配の初期化
        outputs = net(inputs)  # 予測の計算（順伝播）
        loss = loss_fn(outputs, labels)  # 損失関数の計算
        loss.backward()  # 勾配の計算(逆伝播)
        train_loss += loss.item()  # 損失の加算
        acc = (outputs.max(1)[1] == labels).sum() # 正答数の数え上げ
        train_acc += acc.item()  # 正答数の加算
        optimizer.step()  # パラメータの更新
    avg_train_loss = train_loss / len(dl_train.dataset)  # 平均損失の計算
    avg_train_acc = train_acc / len(dl_train.dataset)  # 正答率の計算
        
    net.eval()  # 推論モードに切替
    with torch.no_grad():  # 計算グラフの構築をしないよう設定
        for inputs, labels in dl_valid:
            inputs, labels = inputs.to(device), labels.to(device)  # データをGPUに乗せる
            outputs = net(inputs)  # 予測の計算(順伝播)
            loss = loss_fn(outputs, labels)  # 損失関数の計算
            val_loss += loss.item()  # 損失の加算
            acc = (outputs.max(1)[1] == labels).sum()  # 正答数の数え上げ
            val_acc += acc.item()  # 正答数の加算
    avg_val_loss = val_loss / len(dl_valid.dataset)  # 平均損失の計算
    avg_val_acc = val_acc / len(dl_valid.dataset)  # 正答率の計算
    
    print(('Epoch [{}/{}], train_loss: {train_loss:.5f}, train_acc: {train_acc:.3f}, '
          'val_loss: {val_loss:.5f}, val_acc: {val_acc:.3f}')
          .format(epoch+1, num_epochs, train_loss=avg_train_loss, 
                  train_acc=avg_train_acc, val_loss=avg_val_loss, val_acc=avg_val_acc))

    
# Step 5. 予測の実行
net.eval()  # 推論モードに切替
with torch.no_grad():  # 計算グラフの構築をしないよう設定
    total = 0
    test_acc = 0
    for inputs, labels in dl_test:        
        inputs, labels = inputs.to(device), labels.to(device)  # データをGPUに乗せる
        outputs = net(inputs)  # 予測の計算
        test_acc += (outputs.max(1)[1] == labels).sum().item()  # 正解数の数え上げと加算
        total += labels.size(0)  # テストデータの大きさを取得
    print('Test Accuracy: {} %'.format(100 * test_acc / total)) 